In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import random

Found GPU at: /device:GPU:0


In [ ]:
!wget -O hafez.txt https://raw.githubusercontent.com/amnghd/Persian_poems_corpus/master/normalized/hafez_norm.txt
text = open("hafez.txt","r",encoding="utf-8").read()
print("Total chars :", len(text))

In [4]:
chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Total chars: 36
Number of sequences: 102111


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [5]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)


In [6]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [8]:
epochs = 40
batch_size = 128

for epoch in range(epochs):
    model.fit(x, y, batch_size=batch_size, epochs=1)
    # print()
    # print("Generating text after epoch: %d" % epoch)

    # start_index = random.randint(0, len(text) - maxlen - 1)
    # for diversity in [0.2, 0.5, 1.0, 1.2]:
    #     print("...Diversity:", diversity)

    #     generated = ""
    #     sentence = text[start_index : start_index + maxlen]
    #     print('...Generating with seed: "' + sentence + '"')

    #     for i in range(400):
    #         x_pred = np.zeros((1, maxlen, len(chars)))
    #         for t, char in enumerate(sentence):
    #             x_pred[0, t, char_indices[char]] = 1.0
    #         preds = model.predict(x_pred, verbose=0)[0]
    #         next_index = sample(preds, diversity)
    #         next_char = indices_char[next_index]
    #         sentence = sentence[1:] + next_char
    #         generated += next_char

    #     print("...Generated: ", generated)
    #     print()


798/798 [==============================] - 4s 6ms/step - loss: 1.4761


In [9]:
for i in range(10):
    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print("...Diversity:", diversity)

        generated = ""
        sentence = text[start_index : start_index + maxlen]
        print('...Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            sentence = sentence[1:] + next_char
            generated += next_char

        print("...Generated: ", generated)
        print()

...Diversity: 0.2
...Generating with seed: " اعظم کرد از او کوتاه دست اهرمن
تا ابد م"
...Generated:  ن از میان به در این در این خون دای
به خون دل من از این در می برید از او

از دل و نیازد باد صبحد از آن دل است
به خاک ای که در این در این در او نمی کن
من از من این در من به در این خون دل






با می نه به خون خون به دور ما را به دامن
که در این در این در این قصه داری کرد
من از دل من و نیاز می کند این باد
به خاک نشینی که بر سر خود از در آن چه اند
گر چه ما به دست است و دل بر این خواهد شد
سر ما به جان ک

...Diversity: 0.5
...Generating with seed: " اعظم کرد از او کوتاه دست اهرمن
تا ابد م"
...Generated:  ن همان از او نام و ساز است
دولت بنما که باد آن که می بینم
ز سال هست که از میان می باید است
دلم ز بر خود با سلامت گرد آمد
تیر سلطان من با کالسص و جام می نهادم
به وفات در میان منام دوست
که در این صدر می بر سر به دولت داد
چو ندار به نقش دان دوست در این کن
به کوی فکنه باده مهز می کنند
چون بیا بگرس و به بود صبحم از او

ما در او می زدم در این فتنه سحر در شاهد
ساقیا نیست که من با میان به من با

باد ص

KeyboardInterrupt: ignored

In [10]:
model.save("model.hdf5")